In [3]:
import sys
import numpy as np
import torch
from torch import Tensor, ones, stack, load
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure
import pandas as pd
from torch.nn import Module
from torch.utils.data import DataLoader
from scipy import stats
from pathlib import Path
import wandb
import time
from tesladatano import TeslaDatasetNoStb

# Append PINNFramework
sys.path.append("...") 
import PINNFramework as pf

Was not able to import Horovod. Thus Horovod support is not enabled


In [5]:
# Use cuda if it is available, else use the cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Preprocessing


In [6]:
# # login wandb
# wandb.login()

# # Initialize WandB (optional)
# wandb.init(name='NO_run time stability', 
#            project='Neural_Operator_project',
#            #notes='...', 
#            #tags=['...'],
#            #entity='...'
#            )

# WandB Configurations (optional)
wandb.config.normalize = 1000  
wandb.config.batch_size= 1   
wandb.config.lr = 1e-3
normalize = wandb.config.normalize

# Create instance of the dataset
ds = TeslaDatasetNoStb(device = device, data ="train", normalize = wandb.config.normalize, rel_time = True, diff = "central_diff")

# trainloader
train_loader = DataLoader(ds, batch_size=wandb.config.batch_size,shuffle=True)

model = pf.models.MLP(input_size=6,
                      output_size=1, 
                      hidden_size=100, 
                      num_hidden=4, 
                      lb=ds.lb, 
                      ub=ds.ub,
                      #activation = torch.relu
                      )
model.to(device)

# optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=wandb.config.lr)
criterion = torch.nn.MSELoss()

# # Log the network weight histograms (optional)
# wandb.watch(model)

(445266, 10)


## Function for writing a checkpoint during training


In [ ]:
# function to write a checkpoint
def write_checkpoint(checkpoint_path, epoch, min_mlp_loss, optimizer):
    checkpoint = {}
    checkpoint["epoch"] = epoch
    checkpoint["minimum_pinn_loss"] = min_mlp_loss
    checkpoint["optimizer"] = optimizer.state_dict()
    checkpoint["mlp_model"] = model.state_dict()
    torch.save(checkpoint, checkpoint_path)

## Training of the Neural Operator based on time stability loss


In [ ]:
epochs = 100
min_mlp_loss = np.inf

x_data_plot=[]
y_data_all_plot=[]
y_data_1_plot=[]
y_data_2_plot=[]

# Set fixed random number seed
torch.manual_seed(1234)

begin = time.time()
for epoch in range(epochs):
    # Print epoch
    print(f'Starting epoch {epoch}')
    
    # Set current and total loss value
    current_loss = 0.0
    total_loss = 0.0
    total_loss1 = 0.0
    total_loss2 = 0.0
    
    model.train()   # Optional when not using Model Specific layer
    for i, data in enumerate(train_loader,0):

        x_batch, y_batch, delta_t,rel_t = data

        x_batch = torch.squeeze(x_batch, 0)
        y_batch = torch.squeeze(y_batch, 0)
        delta_t = torch.squeeze(delta_t, 0)
        rel_t = torch.squeeze(rel_t, 0)

        # Ground-truth temperature
        true_temp = x_batch[:,4].detach().clone()

        input0 = x_batch[0].detach().clone()

        # Predicted temperature using model prediction and forward euler method
        #pred_temp = true_temp.detach().clone().to(device)
        pred_temp = torch.zeros(x_batch.shape[0])
        pred_temp[0]=true_temp[0].detach().clone().to(device)

        optimizer.zero_grad()
        prediction = model(x_batch.to(device))
     
        for j in range(0, x_batch.shape[0] - 1):
          input0[4] = torch.tensor(pred_temp[j]).detach().clone()
          #if rel_time == True:
          input0[5] = torch.tensor(rel_t[j]).detach().clone()
          pred = model(input0.to(device))/wandb.config.normalize
          pred_temp[j + 1] = pred_temp[j] + pred*delta_t[j]
 
        loss = criterion(pred_temp.to(device),true_temp.to(device))

        loss.backward()
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        total_loss += loss.item()

        if i % 50 == 49:
            print('Loss after mini-batch %5d: %.8f' %
                  (i + 1, current_loss / 50))
            current_loss = 0.0
    
    Loss = total_loss/(i+1)
    print("Epoch ", epoch, "Total Loss ", Loss)

    x_data_plot.append(epoch)
    y_data_all_plot.append(Loss)
 
    # uncomment for saving the best model and writing checkpoints during training
    # # save best model
    # if min_mlp_loss > Loss:
    #     print(f'Validation Loss Decreased({min_mlp_loss:.6f}--->{Loss:.6f}) \t Saving The Model')
    #     min_mlp_loss = Loss
    #     # Saving State Dict
    #     model_name_path = Path('/nostb/best_model_stb_{}_{}.pt'.format(wandb.run.id, wandb.run.name))
    #     torch.save(model.state_dict(), model_name_path)
    
    ## writing checkpoint
    # if (epoch + 1) % 20 == 0:
    #     checkpoint_path = Path('/nostb/checkpoint_stb_{}_{}_{}.pt'.format(wandb.run.id, wandb.run.name, epoch))
    #     write_checkpoint(checkpoint_path, epoch, min_mlp_loss, optimizer)
        
        
    # # Log the loss and accuracy values at the end of each epoch
    # wandb.log({
    #     "Epoch": epoch,
    #     "Total Loss": Loss,
        # })
end = time.time()
        
print("training time:", end - begin)

## Plot of the loss against epochs

In [ ]:
# Make the plot of Total Loss vs epochs
dpi = 360
figure(figsize=(10, 8), dpi = dpi)
plt.plot(x_data_plot,y_data_all_plot)
plt.title('Training loss vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Total Loss')
plt.show()

## Postprocessing - Load the model

In [26]:
# Import the best model
# PATH = '/nostb/best_model_stb_{}_{}.pt'.format(wandb.run.id, wandb.run.name)
PATH ="/nostb/best_model_stb_3naw721b_NO_run time stability.pt"
model.load_state_dict(torch.load(PATH))
model.eval()

MLP(
  (linear_layers): ModuleList(
    (0): Linear(in_features=6, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Linear(in_features=100, out_features=1, bias=True)
  )
)

## Performance of the model on training data

In [ ]:
# Make a prediction
pred = model(ds.x.float().to(device)) #GPU
pred = pred.detach().cpu().numpy()/normalize

# ground-truth
df_y_tensor_np = ds.y.numpy()/normalize

# Some statistics on the model performance on all of dataset
mae = np.sum(np.abs(pred- df_y_tensor_np).mean(axis=None))
print('MAE:', mae)

mse = ((df_y_tensor_np - pred)**2).mean(axis=None)
print('MSE:', mse)

rel_error = np.linalg.norm(pred - df_y_tensor_np) / np.linalg.norm(df_y_tensor_np)
print('Relative error (%):', rel_error*100)

figure(figsize=(10, 8), dpi= 360)

plt.plot(pred, '--')
plt.plot(df_y_tensor_np, '-')
plt.title('Prediction accuracy on training data')
plt.legend(['Prediction', 'ground-truth'])
plt.xlabel('t-idx')
plt.ylabel('ΔTemp/Δt (°C/s)')
plt.show()

# figure(figsize=(10, 8), dpi= 360)
# #time
# t=ds.t
# plt.plot(t,pred, '--')
# plt.plot(t,df_y_tensor_np, '-')
# plt.title('Prediction accuracy on training data')
# plt.legend(['Prediction', 'ground-truth'])
# plt.xlabel('time / seconds')
# plt.ylabel('ΔTemp/Δt (°C/s)')
# plt.show()


## Function for evaluating the performance of the model on training data

In [28]:

def evaluate(idd,rel_time,diff):
  ds_test = TeslaDatasetNoStb(rel_time = rl, diff = diff,device = device, ID = idd, data = "test")
 

  # Prediction accuracy of the Neural Operator
  print('Prediction accuracy of the Neural Operator (NO)')

  t=ds_test.t
  begin = time.time()
  pred_der = model(ds_test.x.to(device))
  pred_der = pred_der.detach().cpu().numpy()/normalize
  true_der = ds_test.y.numpy()
  end = time.time()
  print("time:", end - begin)

  #MAE
  mae = np.sum(np.abs(pred_der- true_der).mean(axis=None))
  print('MAE:', mae)

  #MSE
  mse = ((true_der - pred_der)**2).mean(axis=None)
  print('MSE:', mse)

  #Relative error
  rel_error = np.linalg.norm(pred_der - true_der) / np.linalg.norm(true_der)
  print('Relative error (%):', rel_error*100)
  plt.figure(figsize = (12, 8))
  plt.plot(t, pred_der, '-', label='Prediction')
  plt.plot(t, true_der, '--', label='Ground-truth')
  plt.title('Prediction accuracy of Neural Operator vs ground-truth for drive-ID = {}'.format(idd))
  plt.xlabel('t (seconds)')
  plt.ylabel('ΔTemp/Δt (°C/s)')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()
  
  print('########################################################')
  

  #3)Forward Euler method with fixed initial env. conditions but with updated 
  #Temperature (and rel time) from the prediction of the model at previous iteration
  #with generated temporally equidistant time steps

  print('Forwad Euler method with fixed initial env conditions')

  rel_t = ds_test.rel_t

  # ground-truth time
  t=ds_test.t
  max_t = t.max()
  t=t.numpy()

  # Ground-truth temperature
  true_temp = ds_test.x[:,4].numpy()

  # Predicted temperature using model prediction and forward euler method
  pred_temp = np.zeros((ds_test.x.shape[0]))
  pred_temp = true_temp.copy()

  # Fixed initial conditions for all environmental conditions
  input = ds_test.x[0].detach().clone()

  # temporally equdistant time steps
  tt = np.linspace(0,max_t,ds_test.x.shape[0])
  step_size=tt[2]-tt[1]

  #ODE
  begin = time.time()

  for i in range(0, ds_test.x.shape[0] - 1):
      input[4] = torch.tensor(pred_temp[i]).detach().clone()
      if rel_time == True:
        input[5] = torch.tensor(rel_t[i]).detach().clone()
      pred = model(input.to(device))
      pred = pred.detach().cpu().numpy()/normalize
      pred_temp[i + 1] = pred_temp[i] + pred*step_size
  end = time.time()

  print("time:", end - begin)

  #MAE
  mae = np.sum(np.abs(pred_temp- true_temp).mean(axis=None))
  print('MAE:', mae)

  #MSE
  mse = ((true_temp - pred_temp)**2).mean(axis=None)
  print('MSE:', mse)

  #Relative error
  rel_error = np.linalg.norm(pred_temp - true_temp) / np.linalg.norm(true_temp)
  print('Relative error (%):', rel_error*100)

  plt.figure(figsize = (12, 8))
  plt.plot(tt, pred_temp, '-', label='Prediction')
  plt.plot(t, true_temp, '--', label='Ground-truth')
  plt.title('Prediction vs ground-truth for drive-ID = {} (temporally equidistant step size)'.format(idd))
  plt.xlabel('t (seconds)')
  plt.ylabel('Temperature (°C)')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()

  print('########################################################')

  #4)Forward Euler method with updated environmental conditions from the dataset at each iteration
  #But with updated temperature from the prediction of the model at previous iteration
  #with true step sizes
  print('Forwad Euler method with updated env conditions from the dataset at each iteration with true step sizes')
  
  # time
  t=ds_test.t
  max_t = t.max()
  t=t.numpy()

  # Ground-truth temperature
  true_temp = ds_test.x[:,4].numpy()

  # Predicted temperature using model prediction and forward euler method
  pred_temp = np.zeros((ds_test.x.shape[0]))
  pred_temp[0] = true_temp[0].copy()


  begin = time.time()
  for i in range(0, ds_test.x.shape[0] - 1):
        input = ds_test.x[i].detach().clone()
        input[4] = torch.tensor(pred_temp[i]).detach().clone()
        pred = model(input.to(device))
        pred = pred.detach().cpu().numpy()/normalize
        pred_temp[i + 1] = pred_temp[i] + pred*(t[i+1]-t[i])
  end = time.time()


  print("time:", end - begin)
  #MAE 
  mae = np.sum(np.abs(pred_temp- true_temp).mean(axis=None))
  print('MAE:', mae)

  #MSE
  mse = ((true_temp - pred_temp)**2).mean(axis=None)
  print('MSE:', mse)

  # Relative error
  rel_error = np.linalg.norm(pred_temp - true_temp) / np.linalg.norm(true_temp)
  print('Relative error (%):', rel_error*100)

  plt.figure(figsize = (12, 8))
  plt.plot(t, pred_temp, '-', label='Prediction')
  plt.plot(t, true_temp, '--', label='Ground-truth')
  plt.title('Prediction (with updated env. conditions) vs ground-truth for drive-ID = {} (true step size)'.format(idd))
  plt.xlabel('t (seconds)')
  plt.ylabel('Temperature (°C)')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()



## Performance of the model on test data

In [ ]:
# Test values = [16,39,47,52,72,81,88]
rl = True
diff = "central_diff"
evaluate(idd=16,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]

evaluate(idd=39,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]

evaluate(idd=47,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]

evaluate(idd=52,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]

evaluate(idd=72,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]

evaluate(idd=81,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]

evaluate(idd=88,rel_time=rl,diff=diff)